## Introduction

In this notebook, we are going to use our previously trained CNN model (refer RSNA 2023 ATD - Baseline 1 [Training](https://www.kaggle.com/code/pankajpansari/rsna-2023-atd-baseline-1-training) notebook) to make prediction on unseen CT scans. We are also going to use this notebook to make our submission for the competition. We make use of the [fastai](https://docs.fast.ai) library.

The advantage of separating the training and inference parts of the pipeline is that we don't need to spend time and compute retraining our model in the cloud. We can simply import the saved model and make predictions. Moreover, this separation enables more flexible development. We can focus on improving training or speeding up inference in an indepenedent way.

Our model makes predictions on individual images. There are sequence of images for each patient. We need to take the image-level predictions and aggregrate them to make prediction for each patient.

## Code

We're going to import the necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
import os, random
from fastai.vision.all import *
from fastai.medical.imaging import *
import shutil
import pydicom
import cv2
import glob
import time
from rsna_2023_atd_metric import score

from PIL import Image

random.seed(1441)

In [ ]:
def standardize_pixel_array(fn):
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    dcm = pydicom.dcmread(fn)
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
#         pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)

    intercept = float(dcm.RescaleIntercept)
    slope = float(dcm.RescaleSlope)
    center = int(dcm.WindowCenter)
    width = int(dcm.WindowWidth)
    low = center - width / 2
    high = center + width / 2    
    
    pixel_array = (pixel_array * slope) + intercept
    pixel_array = np.clip(pixel_array, low, high)

    return pixel_array

In [ ]:
class PILDicom2(PILBase):
    "same as PILDicom but changed pixel array dtype to int32 since uint16 cannot be handled by PIL/PyTorch"
    
    _open_args,_tensor_cls,_show_args = {},TensorDicom,TensorDicom._show_args
    @classmethod
    def create(cls, fn:Path|str|bytes, mode=None)->None:
        "Open a `DICOM file` from path `fn` or bytes `fn` and load it as a `PIL Image`"
        if isinstance(fn,bytes): im = Image.fromarray(pydicom.dcmread(pydicom.filebase.DicomBytesIO(fn)).pixel_array)
        if isinstance(fn,(Path,str)): im = Image.fromarray(standardize_pixel_array(fn).astype(np.int32))
        im.load()
        im = im._new(im.im)
        return cls(im.convert(mode) if mode else im)

In [ ]:
class MultiHeadModel(Module):
    
    def __init__(self, body):
    
        self.body = body
        nf = num_features_model(nn.Sequential(*self.body.children()))

        self.bowel = create_head(nf, 1)
        self.extravasation = create_head(nf, 1)
        self.kidney = create_head(nf, 3)
        self.liver = create_head(nf, 3)
        self.spleen = create_head(nf, 3)
        
    def forward(self, x):
        
        y = self.body(x)
        bowel = self.bowel(y)
        extravasation = self.extravasation(y)
        kidney = self.kidney(y)
        liver = self.liver(y)
        spleen = self.spleen(y)
        return [bowel, extravasation, kidney, liver, spleen]

In [ ]:
class CombinationLoss(Module):
    "Cross entropy loss on multiple targets"
    def __init__(self, func = F.cross_entropy, weights = [2, 6, 3, 3, 3]):
        self.func = func
        self.w = weights
        
    def forward(self, xs, *ys, reduction = 'mean'):
        loss = 0
    
        for i, w, x, y in zip(range(len(xs)), self.w, xs, ys):
            if i < 2:
                loss += w*F.binary_cross_entropy_with_logits(x, y.unsqueeze(1).float(), reduction = reduction)
            else:
                #import pdb;pdb.set_trace()
                loss += w*F.cross_entropy(x, y, reduction = reduction)
        return loss

In [ ]:
from sklearn.metrics import recall_score

class RecallPartial(Metric):
    "Stores predictions and targets on CPU in accumulate to perform final calculations with `func`."
    def __init__(self, a=0, **kwargs):
        self.func = partial(recall_score, average='macro', zero_division=0)
        self.a = a

    def reset(self): self.targs,self.preds = [],[]

    def accumulate(self, learn):
        pred = learn.pred[self.a].argmax(-1)
        targ = learn.y[self.a]
        pred,targ = to_detach(pred),to_detach(targ)
        pred,targ = flatten_check(pred,targ)
        self.preds.append(pred)
        self.targs.append(targ)

    @property
    def value(self):
        if len(self.preds) == 0: return
        preds,targs = torch.cat(self.preds),torch.cat(self.targs)
        return self.func(targs, preds)

    @property
    def name(self): return 'recall_' + str(self.a+1)
    
class RecallCombine(Metric):
    def accumulate(self, learn):
        scores = [learn.metrics[i].value for i in range(3)]
        self.combine = np.average(scores, weights=[2,1,1])

    @property
    def value(self):
        return self.combine

We load the saved model. Note that we want our model to load on the GPU, because we're going to use GPU for inference. To ensure this, we need to set the _cpu_ option to __False__, else the model gets loaded on the CPU by default.

In [ ]:
learn = load_learner('/kaggle/input/rsna-2023-atd-baseline-1-training/model.pt', cpu = False)

The original train/test CT scan images are in DICOM format, but we're going to use the PNG format for making predictions.

In [ ]:
TEST_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/'
NUM_SCANS = 256

print('Number of test patients:', len(os.listdir(TEST_PATH)))

In the following, we loop over the patients. For each patient, we convert the DICOM images to rescaled PNG images and save them. Then, we make a dataloader using these images and make predictions. We keep the batch size relatively high to make good use of GPU. We save the predictions in a list and delete the images since we don't need them further.

In [ ]:
def merge_arr(a, b):
    return np.concatenate((a, b.numpy()), axis = 0)

In [ ]:
patients = random.choices(os.listdir(TEST_PATH), k = 10)

bowel_preds, extrav_preds = np.array([]).reshape(0), np.array([]).reshape(0)
kidney_preds, liver_preds, spleen_preds = np.array([]).reshape(0, 3), np.array([]).reshape(0, 3), np.array([]).reshape(0, 3)
fnames_list = []

start = time.time()
sigm = torch.nn.Sigmoid()
softm = torch.nn.Softmax(dim = 1)

for idx, patient in enumerate(patients):
    
    series = os.listdir(os.path.join(TEST_PATH, patient))
    
    for s in series:
        
        files = random.choices(os.listdir(os.path.join(TEST_PATH, patient, s)), k = NUM_SCANS)
        files = [os.path.join(TEST_PATH, patient, s, f) for f in files]
        test_dl = learn.dls.test_dl(files, with_labels = False, device = 'cuda', bs = 256)

        preds = learn.get_preds(dl = test_dl)[0]
        
        bowel_preds = merge_arr(bowel_preds, sigm(preds[0]).squeeze(-1))
        extrav_preds = merge_arr(extrav_preds, sigm(preds[1]).squeeze(-1))
        kidney_preds = merge_arr(kidney_preds, softm(preds[2]))
        liver_preds = merge_arr(liver_preds, softm(preds[3]))
        spleen_preds = merge_arr(spleen_preds, softm(preds[4]))
       
        fnames_list.append(files)

    if (idx + 1) % 100 == 0:
        end = time.time()
        print(f'{idx + 1} patients processed.')
        print(f'Time elapsed: {end - start} ')
        print(f'Avg time per patient: {(end - start)/(idx + 1)}')

We convert the prediction list to a numpy array.

In [ ]:
from itertools import chain
fnames_list = list(chain.from_iterable(fnames_list))

From the predictions, we derive the individual probabilities for different conditions. Our predictions are the probabilities of injury in bowel, extravasation, liver, kidney, and spleen. The probability of the organs being healthy is naturally 1 - probability of injury. We take the probability of liver, kidney, and  spleen injury and divide them equally between low and high types.

In [ ]:
test_files_probs = pd.DataFrame()

test_files_probs['fname'] = pd.Series(fnames_list, dtype = 'string')

test_files_probs['bowel_healthy'] = pd.Series(1 - bowel_preds)
test_files_probs['bowel_injury'] = pd.Series(bowel_preds)
test_files_probs['extravasation_healthy'] = pd.Series(1 - extrav_preds)
test_files_probs['extravasation_injury'] = pd.Series(extrav_preds)
test_files_probs['kidney_healthy'] = pd.Series(kidney_preds[:, 0])
test_files_probs['kidney_low'] = pd.Series(kidney_preds[:, 1])
test_files_probs['kidney_high'] = pd.Series(kidney_preds[:, 2])
test_files_probs['liver_healthy'] = pd.Series(liver_preds[:, 0])
test_files_probs['liver_low'] = pd.Series(liver_preds[:, 1])
test_files_probs['liver_high'] = pd.Series(liver_preds[:, 2])
test_files_probs['spleen_healthy'] = pd.Series(spleen_preds[:, 0])
test_files_probs['spleen_low'] = pd.Series(spleen_preds[:, 1])
test_files_probs['spleen_high'] = pd.Series(spleen_preds[:, 2])

#test_files_probs

We add a new column _'patient_id'_ derived from file name. This will be helpful in aggregating the predictions from image-level to patient-level.

In [ ]:
patient_id_list = []
for idx, fname in enumerate(test_files_probs['fname']):
    patient_id_list.append(fname.split('/')[-3])
    
test_files_probs['patient_id'] = pd.Series(patient_id_list, dtype = 'string')

test_files_probs = test_files_probs.drop('fname', axis = 1)

We simply take the mean over the predictons of all the CT scan images for each patient to make the prediction for each patient.

In [ ]:
patient_probs = test_files_probs.groupby('patient_id').mean()

patient_probs.insert(0, 'patient_id', patient_probs.index)

patient_probs = patient_probs.reset_index(drop = True)
patient_probs.head()

Finally, we write the predictions for all patients to _submission.csv_ file and submit our notebook.

In [ ]:
patient_probs.to_csv('submission.csv', header = True, index = False)

### Computing Evaluation Metric on a Dataset

In [ ]:
solution = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
solution.patient_id = solution.patient_id.astype(str)

solution = solution[solution.patient_id.isin(patient_probs.patient_id)]

In [ ]:
solution['bowel_weight'] = pd.Series(np.maximum(1*solution.bowel_healthy.to_numpy(), 2*solution.bowel_injury.to_numpy()))
solution['extravasation_weight'] = pd.Series(np.maximum(1*solution.extravasation_healthy.to_numpy(), 6*solution.extravasation_injury.to_numpy()))
solution['kidney_weight'] = pd.Series(np.maximum.reduce([1*solution.kidney_healthy.to_numpy(), 2*solution.kidney_low.to_numpy(), 4*solution.kidney_high.to_numpy()]))
solution['spleen_weight'] = pd.Series(np.maximum.reduce([1*solution.spleen_healthy.to_numpy(), 2*solution.spleen_low.to_numpy(), 4*solution.spleen_high.to_numpy()]))
solution['liver_weight'] = pd.Series(np.maximum.reduce([1*solution.liver_healthy.to_numpy(), 2*solution.liver_low.to_numpy(), 4*solution.liver_high.to_numpy()]))
solution['any_injury_weight'] = pd.Series([6]*solution.shape[0])

In [ ]:
solution

In [ ]:
#import pdb; pdb.set_trace()
score(solution, patient_probs, 'patient_id')

In [ ]:
patient_probs